In [49]:
import numpy as np
import cv2
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from sklearn.model_selection import train_test_split

from tensorflow.python.keras.utils.np_utils import to_categorical

In [50]:
DATA_PATH = os.path.join('MP_Data') 

actions = np.array(['homem','obrigado', 'oi'])

max_frames_per_video = 200
numeros_videos_por_action = 3
label_map = {label:num for num, label in enumerate(actions)}

In [51]:
sequences, labels = [], []

for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(max_frames_per_video):
            file_path = os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num))
            
            try:
                res = np.load(file_path)
            except FileNotFoundError:
                array_shape = (1662,)
                zeros_array = np.zeros(array_shape)
                window.append(zeros_array)
            else:
                window.append(res)
        
        sequences.append(window)
        labels.append(label_map[action])


In [58]:
y = np.array(labels)
X = np.array(sequences)
y = to_categorical(labels).astype(int)

C:\Users\Marcelo\AppData\Local\Temp\ipykernel_12312\2102816456.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  X = np.array(sequences)


In [39]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.30)


In [40]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [41]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)
actions.shape

(3,)

In [42]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(200,258)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [46]:
from tensorflow.keras.layers import Flatten

# Define your model
model = Sequential()
model.add(LSTM(256, return_sequences=True, activation='relu', input_shape=(200, 258)))
model.add(LSTM(64, return_sequences=True, activation='relu'))
model.add(LSTM(128, return_sequences=False, activation='relu'))
model.add(Dense(256, activation='tanh'))
model.add(Dense(64, activation='relu'))

# Add a Flatten layer to reshape the output
model.add(Flatten())

# Add a Dense layer with 3 units and softmax activation for classification
model.add(Dense(3, activation='softmax'))

In [47]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [48]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).